In [116]:
import pymysql
# from sshtunnel import SSHTunnelForwarder
import mysql.connector
import pandas as pd
from tqdm import tqdm

## Connecting to the DB instance

In [25]:
host = 'database.c5qerzx5gva8.us-east-2.rds.amazonaws.com'
user = 'admin'
password = 'Admin123!'
database = 'sys'
port = 3306

In [92]:
db = pymysql.connect(host=host, user=user, password=password)

In [93]:
cursor = db.cursor()

## Creating DB & tables

In [97]:
sql = '''create database hse'''
cursor.execute(sql)

1

In [98]:
cursor.connection.commit()

In [99]:
sql = '''use hse'''
cursor.execute(sql)

0

In [101]:
len('2021-12-05')

10

In [245]:
sql = '''drop table rankings'''
cursor.execute(sql)

0

In [139]:
sql = '''
create table housing_faq (
   event_date CHAR(10),
   question VARCHAR(65534),
   answer VARCHAR(65534)
)
'''
cursor.execute(sql)

0

In [153]:
sql = '''
create table costs (
   event_date CHAR(10),
   which VARCHAR(65534),
   programme VARCHAR(65534),
   tuition VARCHAR(65534)
)
'''
cursor.execute(sql)

0

In [246]:
sql = '''
create table rankings (
   event_date CHAR(10),
   ranking VARCHAR(65534),
   subject VARCHAR(65534),
   place_world VARCHAR(65534),
   place_russia VARCHAR(65534)
)
'''
cursor.execute(sql)

0

In [240]:
sql = '''
create table programmes (
   event_date CHAR(10),
   programme VARCHAR(65534),
   faculty VARCHAR(65534),
   link VARCHAR(65534),
   type VARCHAR(65534),
   department VARCHAR(65534),
   location VARCHAR(65534),
   duration VARCHAR(65534),
   schedule VARCHAR(65534),
   language VARCHAR(65534)
)
'''
cursor.execute(sql)

0

In [241]:
sql = '''show tables'''
cursor.execute(sql)
cursor.fetchall()

(('costs',), ('housing_faq',), ('programmes',), ('rankings',))

## Inserting

In [152]:
from parsing_funcs import *

In [137]:
hfaq = parse_housing_faq()

hfaq['date'] = '2021-12-05'
hfaq = hfaq[['date', 'question', 'answer']]
for col in ['question', 'answer']:
    hfaq[col] = hfaq[col].apply(lambda x: x.replace("'","`"))
hfaq.head()

,date,question,answer
0,2021-12-05,How can I apply for accommodation?,You will need to fill out a form in your onlin...
1,2021-12-05,Will I sign a formal rental agreement?,"Yes, you will. Each student signs a rental agr..."
2,2021-12-05,Can I contact the Student Housing Office manag...,Yes. If you have any questions about your acco...
3,2021-12-05,How will I pay for accommodation? Via the univ...,If the contract (rental or accommodation contr...
4,2021-12-05,Will there be a security deposit?,All accommodation options require a security d...


In [127]:
def get_insert_hfaq_line_query(d, q, a, db='hse', table='housing_faq'):
    query = f"""
        INSERT INTO {db}.{table} ( event_date, question, answer )
           VALUES
           ( '{d}', '{q}', '{a}' );
    """
    return query

In [128]:
sql = get_insert_hfaq_line_query(hfaq['date'][0], hfaq['question'][0], hfaq['answer'][0])
cursor.execute(sql)
db.commit()

In [141]:
sql = '''select count(*) from housing_faq'''
cursor.execute(sql)
cursor.fetchall()

((0,),)

In [142]:
error_idx = []
for i in tqdm(range(hfaq.shape[0])):
    try:
        sql = get_insert_hfaq_line_query(hfaq['date'][i], hfaq['question'][i], hfaq['answer'][i])
        cursor.execute(sql)
        db.commit()
    except Exception as e:
        print(i, e)
        error_idx.append(i)
        pass

100%|██████████| 37/37 [00:11<00:00,  3.23it/s]


In [145]:
sql = '''select count(*) from housing_faq'''
cursor.execute(sql)
cursor.fetchall()[0][0]

37

### housing

In [147]:
def insert_housing_faq(date, link='https://www.hse.ru/en/sho/', table='housing_faq'):
    
    # parsing
    hfaq = parse_housing_faq(link=link)
    hfaq['date'] = date
    hfaq = hfaq[['date', 'question', 'answer']]
    
    # replacing single quotes
    for col in ['question', 'answer']:
        hfaq[col] = hfaq[col].apply(lambda x: x.replace("'","`"))
    
    # one line insert query 
    def get_insert_hfaq_line_query(d, q, a, db='hse', table='housing_faq'):
        query = f"""
            INSERT INTO {db}.{table} ( event_date, question, answer )
               VALUES
               ( '{d}', '{q}', '{a}' );
        """
        return query
    
    # inserting
    error_idx = []
    for i in tqdm(range(hfaq.shape[0])):
        try:
            sql = get_insert_hfaq_line_query(
                hfaq['date'][i], 
                hfaq['question'][i], 
                hfaq['answer'][i], 
                table=table
            )
            cursor.execute(sql)
            db.commit()
        except Exception as e:
            print(i, e)
            error_idx.append(i)
            pass
        
    # checking how many rows are uploaded
    sql = f"""select count(*) from {table} where event_date = '{date}'"""
    cursor.execute(sql)
    print(f"Uploaded {cursor.fetchall()[0][0]} records out of {hfaq.shape[0]}")
        
    return hfaq, error_idx

In [150]:
sql = '''select min(event_date), max(event_date) from housing_faq'''
cursor.execute(sql)
cursor.fetchall()

(('2021-12-05', '2021-12-05'),)

In [151]:
tab, error_idx = insert_housing_faq(date='2021-12-04')

100%|██████████| 37/37 [00:12<00:00,  3.06it/s]

Uploaded 74 records out of 37


In [250]:
sql = '''select event_date, count(*) from housing_faq group by event_date'''
cursor.execute(sql)
cursor.fetchall()

(('2021-12-05', 37), ('2021-12-04', 37))

### costs

In [157]:
def insert_costs(date, link='https://admissions.hse.ru/en/graduate-apply/fees', table='costs'):
    
    stats = {}
    # for each costs type
    for which in ['fees', 'living']:
        # parsing
        costs = parse_costs(link=link, which=which)
        costs.columns = ['programme', 'tuition']        
        costs['date'] = date
        costs = costs[['date', 'programme', 'tuition']]

        # replacing single quotes
        for col in ['programme', 'tuition']:
            costs[col] = costs[col].apply(lambda x: x.replace("'","`"))

        # one line insert query 
        def get_insert_costs_line_query(d, q, a, db='hse', table='costs'):
            query = f"""
                INSERT INTO {db}.{table} ( event_date, programme, tuition )
                   VALUES
                   ( '{d}', '{q}', '{a}' );
            """
            return query

        # inserting
        error_idx = []
        for i in tqdm(range(costs.shape[0])):
            try:
                sql = get_insert_costs_line_query(
                    costs['date'][i], 
                    costs['programme'][i], 
                    costs['tuition'][i], 
                    table=table
                )
                cursor.execute(sql)
                db.commit()
            except Exception as e:
                print(i, e)
                error_idx.append(i)
                pass

        # checking how many rows are uploaded
        sql = f"""select count(*) from {table} where event_date = '{date}'"""
        cursor.execute(sql)
        print(f"Uploaded {cursor.fetchall()[0][0]} records out of {costs.shape[0]}")
        
        stats[which] = [costs, error_idx]
    
    return stats

In [158]:
c = insert_costs('2021-12-05')

100%|██████████| 39/39 [00:13<00:00,  2.80it/s]


Uploaded 0 records out of 39


100%|██████████| 39/39 [00:12<00:00,  3.15it/s]

Uploaded 0 records out of 39


In [251]:
sql = '''select event_date, count(*) from costs group by event_date'''
cursor.execute(sql)
cursor.fetchall()

(('2021-12-05', 78),)

### rankings

In [247]:
def insert_rankings(date, link='https://strategy.hse.ru/en/rating/', table='rankings'):
    
    # parsing
    ranking = parse_rankings(link=link)
    ranking.columns = ['ranking', 'subject', 'place_world', 'place_russia']
    ranking['date'] = date
    ranking = ranking[['date', 'ranking', 'subject', 'place_world', 'place_russia']]
    
    # replacing single quotes
    def replace_single_quotes(x):
        try:
            return x.replace("'","`")
        except AttributeError: # None
            return ''
    for col in ['ranking', 'subject', 'place_world', 'place_russia']:
        ranking[col] = ranking[col].apply(replace_single_quotes)
    
    # one line insert query 
    def get_insert_rankings_query(d, r, s, pw, pr, db='hse', table='rankings'):
        query = f"""
            INSERT INTO {db}.{table} ( event_date, ranking, subject, place_world, place_russia )
               VALUES
               ( '{d}', '{r}', '{s}', '{pw}', '{pr}' );
        """
        return query
    
    # inserting
    error_idx = []
    for i in tqdm(range(ranking.shape[0])):
        try:
            sql = get_insert_rankings_query(
                ranking['date'][i], 
                ranking['ranking'][i], 
                ranking['subject'][i], 
                ranking['place_world'][i], 
                ranking['place_russia'][i],
                table=table
            )
            cursor.execute(sql)
            db.commit()
        except Exception as e:
            print(i, e)
            error_idx.append(i)
            pass
        
    # checking how many rows are uploaded
    sql = f"""select count(*) from {table} where event_date = '{date}'"""
    cursor.execute(sql)
    print(f"Uploaded {cursor.fetchall()[0][0]} records out of {ranking.shape[0]}")
        
    return ranking, error_idx

In [248]:
tab, error_idx = insert_rankings(date='2021-12-04')

 69%|██████▉   | 38/55 [00:10<00:04,  3.57it/s]

38 38


100%|██████████| 55/55 [00:15<00:00,  3.64it/s]

Uploaded 54 records out of 55


In [252]:
sql = '''select event_date, count(*) from rankings group by event_date'''
cursor.execute(sql)
cursor.fetchall()

(('2021-12-04', 54),)

### programmes

In [ ]:
def insert_programmes(date, link='https://www.hse.ru/en/education/magister/', table='programmes'):
    
    # parsing
    programmes = parse_programmes(link=link)
    programmes.columns = ['programme', 'faculty', 'link', 'type', 'department', 'location', 
                       'duration', 'schedule', 'language']
    programmes['date'] = date
    programmes = programmes[['date', 'programme', 'faculty', 'link', 'type', 'department', 'location', 
                       'duration', 'schedule', 'language']]
    
    # replacing single quotes
    def replace_single_quotes(x):
        try:
            return x.replace("'","`")
        except AttributeError: # None
            return ''
    for col in ['programme', 'faculty', 'link', 'type', 'department', 'location', 
                       'duration', 'schedule', 'language']:
        programmes[col] = programmes[col].apply(replace_single_quotes)
    
    # one line insert query 
    def get_insert_programmes_query(d, f, l, t, dep, loc, dur, sch, lan,  db='hse', table='programmess'):
        query = f"""
            INSERT INTO {db}.{table} ( event_date, faculty, link, type, department, location, duration, schedule, language )
               VALUES
               ( '{d}', '{f}', '{l}', '{t}', '{dep}', '{loc}', '{dur}', '{sch}', '{lan}' );
        """
        return query
    
    # inserting
    error_idx = []
    for i in tqdm(range(programmes.shape[0])):
        try:
            sql = get_insert_programmes_query(
                programmes['date'][i], 
                programmes['faculty'][i], 
                programmes['link'][i], 
                programmes['type'][i], 
                programmes['department'][i],
                programmes['location'][i],
                programmes['duration'][i],
                programmes['schedule'][i],
                programmes['language'][i],
                table=table
            )
            cursor.execute(sql)
            db.commit()
        except Exception as e:
            print(i, e)
            error_idx.append(i)
            pass
        
    # checking how many rows are uploaded
    sql = f"""select count(*) from {table} where event_date = '{date}'"""
    cursor.execute(sql)
    print(f"Uploaded {cursor.fetchall()[0][0]} records out of {programmes.shape[0]}")
        
    return programmes, error_idx

In [ ]:
tab, error_idx = insert_programmes(date='2021-12-04')

In [253]:
sql = '''select event_date, count(*) from programmes group by event_date'''
cursor.execute(sql)
cursor.fetchall()

(('2021-12-04', 283),)